# Tner Trainer 

trainer를 활용해 fine-tuning가능

Task에 맞는 데이터셋 or 직접 라벨링한 데이터셋을 활용한다면 더 좋은 결과를 얻을 수 있을 것이라 예상

In [59]:
import tner
import pandas as pd
from nltk.tokenize import sent_tokenize
import string
import numpy as np

In [5]:
trainer = tner.TrainTransformersNER(checkpoint_dir='./ckpt_merged', dataset=["ontonotes5", "conll2003"], transformers_model="xlm-roberta-base")

2022-07-11 10:06:26 INFO     *** initialize network ***
2022-07-11 10:06:26 INFO     create new checkpoint
2022-07-11 10:06:26 INFO     checkpoint: ./ckpt_merged_ba09202a5419c735668d527d68bebfa4
2022-07-11 10:06:26 INFO      - [arg] dataset: ['ontonotes5', 'conll2003']
2022-07-11 10:06:26 INFO      - [arg] transformers_model: xlm-roberta-base
2022-07-11 10:06:26 INFO      - [arg] random_seed: 1234
2022-07-11 10:06:26 INFO      - [arg] lr: 1e-05
2022-07-11 10:06:26 INFO      - [arg] total_step: 5000
2022-07-11 10:06:26 INFO      - [arg] warmup_step: 700
2022-07-11 10:06:26 INFO      - [arg] weight_decay: 1e-07
2022-07-11 10:06:26 INFO      - [arg] batch_size: 32
2022-07-11 10:06:26 INFO      - [arg] max_seq_length: 128
2022-07-11 10:06:26 INFO      - [arg] fp16: False
2022-07-11 10:06:26 INFO      - [arg] max_grad_norm: 1.0
2022-07-11 10:06:26 INFO      - [arg] lower_case: False


In [ ]:
trainer.train(monitor_validation=True) #인코딩 에러가 날시 수정해주면 정상 작동

# START

# Pretrained-model 불러오기

In [10]:
#모든 데이터셋 사전학습된 모델 (이게 성능이 더좋음 채택)
model = tner.TransformersNER("asahi417/tner-xlm-roberta-base-all-english")

2022-07-11 10:09:40 INFO     *** initialize network ***


Downloading:   0%|          | 0.00/3.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/211 [00:00<?, ?B/s]

In [1]:
#fine-tuning 진행한 모델
classifier = tner.TransformersNER('./ckpt_merged_openote5')

2022-05-31 16:02:55 INFO     NumExpr defaulting to 4 threads.
2022-05-31 16:02:56 INFO     *** initialize network ***


# 데이터 불러오기 및 간단한 전처리

In [11]:
import os
folders=os.listdir('C:/Users/Quantec/quantec/data/week_news')

In [12]:
df_all_news=pd.DataFrame()

for files in folders:
    df=pd.read_csv('C:/Users/Quantec/quantec/data/week_news/'+files)
    df_all_news=pd.concat([df_all_news,df])

In [13]:
df_all_news=df_all_news.drop_duplicates(['date','contents'],keep='first') #중복제거

In [15]:
#뉴스 데이터 중 특정 날짜만 뽑아내기 위함
df_all_news['date']=df_all_news.date.astype(str)

df_all_news['date']=pd.to_datetime(df_all_news['date'])

df_all_news=df_all_news[df_all_news.date.between('2022-05-01','2022-05-10')].sort_values('date') #테스트를 위해 날짜 짧게 조정

df_all_news.reset_index(drop=True,inplace=True)

text=df_all_news

In [16]:
text.head(5)

,date,symbol,link,contents,summary,keywords_pki,sentiment_score,sentiment_list,name,provider
0,2022-05-06,IHG,https://finance.yahoo.com/news/ihg-hotels-reso...,InterContinental Hotels Group PLC (NYSE: IHG) ...,InterContinental Hotels Group PLC (NYSE: IHG) ...,"InterContinental_Hotels_Group,IHG,Hotels,Resor...",0.0,{},NaN,NaN
1,2022-05-06,C,https://finance.yahoo.com/news/citigroup-court...,Citigroup Inc. wants a bigger piece of that de...,Citigroup Inc. wants a bigger piece of that de...,"Citigroup,piece,Wall,Street,buyout,bank,busine...",0.0,"{'-crisis': 1, '+opportunity': 1}",NaN,NaN
2,2022-05-06,CI,https://finance.yahoo.com/news/cigna-reports-q...,Health insurer Cigna Corp raised its annual ad...,Health insurer Cigna Corp raised its annual ad...,"health,insurer,Cigna,profit,forecast,decline,C...",0.0,"{'-decline': 1, '+strength': 1}",NaN,NaN
3,2022-05-06,ENB,https://finance.yahoo.com/news/enbridge-first-...,Canadian energy pipeline company Enbridge Inc ...,Canadian energy pipeline company Enbridge Inc ...,"canadian,energy,pipeline,company,Enbridge,rise...",1.0,{'+opportunity': 1},NaN,NaN
4,2022-05-06,DB,https://finance.yahoo.com/news/italy-appeals-c...,An Italian appeals court on Friday acquitted a...,An Italian appeals court on Friday acquitted a...,"italian,appeal,court,defendant,Deutsche_Bank,N...",-4.0,"{'-defendant': 1, '-loss': 1, '-verdict': 2}",NaN,NaN


In [98]:
# 주차 단위로 구할 때 활용
# text=pd.read_csv('C:/Users/Quantec/quantec/data/20220623_news.csv')

# text=text.drop_duplicates(['date','contents'],keep='first')

# text['date']=text.date.astype(str)

# text['date']=text['date'].apply(lambda x : x[:8])


# text['date']=pd.to_datetime(text['date'])

# text=text[text.date.between('2022-06-17','2022-06-30')].sort_values('date')

# text.reset_index(drop=True,inplace=True)


# 뉴스 문서 Sent_tokenize로 문장단위 토큰화

In [17]:

from tqdm import tqdm
sentence_list=[]

index_list=[]
for i in tqdm(range(len(text.contents))):
    count=len(sent_tokenize(text.contents[i]))
    for k in range(count):
        word=sent_tokenize(text.contents[i])[k].split()
        sent=sent_tokenize(text.contents[i])[k]
        sentence_list.append(sent)
        index_list.append(i)

100%|██████████| 472/472 [00:13<00:00, 33.82it/s]


### NER 추출

문장내에서 Ticker만 추출해야 하기 때문에 기본 모델 구조에서 output부분을 수정했음

entity -> organzation만 추출

데이터 단위가 문장단위 이기 때문에 tner tokenizer를 그대로 사용


In [18]:
#메모리 에러로 400개씩만 input으로 predict.
output_list=[]
for i in tqdm(range(0,len(sentence_list),400)):
    s_text=sentence_list[i:i+400]
    output=model.predict(s_text)
    output_list.append(output)
    
#outputs=classifier.predict(test)


100%|██████████| 25/25 [05:45<00:00, 13.81s/it]


In [19]:
result=sum(output_list,[])

In [20]:
entity_list=[]
for i in range(len(result)):
    entity=list(result[i].values())[0]
    entity_list.append(entity)
    
symbol_news=pd.DataFrame([index_list,sentence_list,entity_list]).T
symbol_news.columns=['news_label','contents','entity_list']

In [21]:
symbol_news.head(5)

,news_label,contents,entity_list
0,0,InterContinental Hotels Group PLC (NYSE: IHG) ...,"[InterContinental Hotels Group PLC, IHG Hotels..."
1,0,Rise in demand for hotel stays in the U.S. spr...,[Reuters]
2,0,"Q1 RevPAR, or revenue per available room, was ...",[]
3,0,The company noted Americas and EMEAA saw seque...,"[Americas, EMEAA]"
4,0,"Meanwhile, China's trading in March was impact...",[]


In [26]:
#entity가 추출되지 않은 문장은 제거
df_list=[]
for i in range(symbol_news.shape[0]):
    symbol_list=entity_list[i]
    if len(symbol_list) > 0:
        df_list.append(symbol_news.iloc[i])
        

In [27]:
ner_df=pd.DataFrame(df_list)

In [28]:
ner_df.reset_index(inplace=True,drop=True)

In [31]:
ner_df.to_csv('sentiment_05_06.csv',encoding='utf-8-sig') #한번 저장

In [32]:
#전처리
# ner_df.entity_list=ner_df.entity_list.astype(str)
# ner_df.entity_list=ner_df['entity_list'].str.replace('[',"",regex=True)
# ner_df.entity_list=ner_df['entity_list'].str.replace(']',"",regex=True)
# ner_df.entity_list=ner_df['entity_list'].str.replace('"',"",regex=True)
# ner_df.entity_list=ner_df['entity_list'].str.replace('\'',"",regex=True)

# ner_df.entity_list=ner_df.entity_list.str.replace(","," ",regex=True)
# ner_df.entity_list=ner_df.entity_list.str.replace("  "," ",regex=True)

In [656]:
# quotes=pd.read_csv('data/US_topics_quotes.csv')

In [657]:
# for i in range(quotes.summary.shape[0]):
#     quotes.loc[i,'summary']=quotes.summary[i].replace('company',quotes.symbol[i])
#     quotes.loc[i,'summary']=quotes.summary[i].split('\'execs\'')[0]
#     quotes.loc[i,'summary']=quotes.summary[i].replace('],','')
#     sub=quotes.summary[i].split('\'details\': [')
#     sub[0]=sub[0].rstrip()
#     quotes.loc[i,'summary']=', '.join(sub)
#     quotes.loc[i,'summary']=quotes.summary[i].rstrip()
#     quotes.loc[i,'summary']=quotes.summary[i].split(': [')[1]
#     #quotes.summary[i]+']}'
# for i in range(quotes.summary.shape[0]):
#     quotes.loc[i,'summary']=quotes.summary[i].split('\'execs\'')[0]+']}'
    
# for i in range(quotes.summary.shape[0]):
#     quotes.loc[i,'summary']=quotes.summary[i].replace('],','')
    
#quotes.to_csv('data/US_topic_quotes_gaja.csv')

# 추출된 organzation word-> Ticker 변환

우선 기존에 정의한 ticker 사전에 추가로 filter 해야하는 경우의 수 추가

모든 organzation을 ticker로 변환하지 않는 이유는 사전 정의한 리스트에서 제외 대상이기 때문

In [30]:
#사전 정의한 ticker 사전 
quotes=pd.read_csv('data/US_topic_quotes_gaja.csv')
quotes.drop('Unnamed: 0', axis=1,inplace=True)


for i in range(quotes.shape[0]):
    quotes.loc[i,'summary']=quotes.summary[i].replace("\"","\'")

In [31]:
www=pd.read_csv('data/US_topics_quotes.csv')

In [34]:
www

,region,asset,listed,name,symbol,update_dttm,last_page,summary
0,US,Topics,US,"Agilent Technologies, Inc.",A,20220509 141504,NaN,"{'company': ['Agilent Technologies, Inc.'], 'd..."
1,US,Topics,US,Alcoa Corporation,AA,20220509 153715,NaN,"{'company': ['Alcoa Corporation'], 'details': ..."
2,US,Topics,US,American Airlines Group Inc.,AAL,20220509 163844,NaN,"{'company': ['American Airlines Group Inc.'], ..."
3,US,Topics,US,"Advance Auto Parts, Inc.",AAP,20220510 160456,NaN,"{'company': ['Advance Auto Parts, Inc.'], 'det..."
4,US,Topics,US,Apple Inc.,AAPL,20220511 150807,NaN,"{'company': ['Apple Inc.'], 'details': ['Mac',..."
...,...,...,...,...,...,...,...,...
931,US,Topics,US,Zynga Inc.,ZNGA,20220512 113031,NaN,"{'company': ['Zynga Inc.', 'Zynga'], 'details'..."
932,US,Topics,US,China Southern Airlines Company Limited,ZNH,20220513 092955,NaN,{'company': ['China Southern Airlines Company ...
933,US,Topics,US,"Zscaler, Inc.",ZS,20220509 142429,NaN,"{'company': ['Zscaler, Inc.', 'Zscaler'], 'det..."
934,US,Topics,US,ZTO Express (Cayman) Inc.,ZTO,20220509 154647,NaN,"{'company': ['ZTO Express', 'Cayman'], 'detail..."


In [36]:
#이름 제거
www['name']=www['name'].apply(lambda x : ''.join([k for k in x if k not in string.punctuation]))

In [37]:
#name + , . 추가
all_list=[]
for i in range(www.shape[0]):
    www_list=[]
    aa=www.name[i]+'.'
    corp_sum=www.name[i].split()[0:len(www.name[i].split())-1]
    corp_sum_=' '.join(corp_sum)+' '+'Corp'
    corp_sum__=' '.join(corp_sum)+' '+'Corp.'
    co_sum_=' '.join(corp_sum)+' '+'Co'
    co_sum__=' '.join(corp_sum)+' '+'Co.'
    Ltd_sum_=' '.join(corp_sum)+' '+'Ltd'
    Ltd_sum__=' '.join(corp_sum)+' '+'Ltd.'
    www_list.append(corp_sum_)
    www_list.append(corp_sum__)
    www_list.append(co_sum_)
    www_list.append(co_sum__)
    www_list.append(Ltd_sum_)
    www_list.append(Ltd_sum__)
            
    if len(www.name[i].split()) == 2 :
        ee=', '.join(www.name[i].split())
        ee_=ee+'.'
        www_list.append(ee)
        www_list.append(ee_)
    elif len(www.name[i].split()) >2 :
        ss=www.name[i].split()[len(www.name[i].split())-2:len(www.name[i].split())]
        et=', '.join(ss)
        rq=www.name[i].replace(' '.join(www.name[i].split()[len(www.name[i].split())-2:len(www.name[i].split())]),et)
        rq_=rq+'.'
        www_list.append(rq)
        www_list.append(rq_)
    www_list.append(www.name[i])
    www_list.append(aa)


    if len(www.name[i].split()) == 2 :
        one=www.name[i].split()[0]
        www_list.append(one)
        
    elif len(www.name[i].split()) > 2:
        for count in range(len(www.name[i].split())):
            bi=www.name[i].split()[0:count]
            bi_sum=' '.join(bi)
            
            if len(bi_sum) > 0:
                www_list.append(bi_sum)
                
    all_list.append(www_list)


In [38]:
st_list=[]
for i in range(quotes.shape[0]):
    st=quotes.summary[i].split("\', ")
    for j in range(len(st)):
        st[j]=st[j].replace("\'",'')
    st.extend(all_list[i])
    st.append(quotes.symbol[i])
    st_list.append(st)
    
    

In [39]:
#중복제거
for i in range(len(st_list)):
    new_list=list(set(st_list[i]))
    st_list[i]=new_list

In [40]:
#AAL -> American을 제외한 나머지 ticker에서 American 제거
for i in range(3,936):
    try:
        st_list[i].remove('American')
    except:
        pass
    
    

In [41]:
symbol_dict=dict(zip(quotes.symbol,st_list))

In [52]:
symbol_dict['AAPL']

['Apple Co',
 'Apple Co.',
 'iPod',
 'Mac',
 'Apple TV',
 'Apple TV+',
 'iCloud',
 'Siri',
 'iPad',
 'Apple Ltd.',
 'HomePod',
 'iOS',
 'Apple Inc.',
 'AppleCare',
 'Apple Inc',
 'AppStore',
 'Apple Home',
 'macOS',
 'Apple Watch',
 'AirPods',
 'Apple',
 'Apple Ltd',
 'Apple, Inc.',
 'iPhone',
 'Apple Corp',
 'Apple, Inc',
 'AAPL',
 'AirTag',
 'iPadOS',
 'Apple Corp.']

In [168]:
#ner_df=pd.read_csv('sentiment_05_06.csv',encoding='utf-8-sig')#위에서 생성된 ner_df 활용하면 됨.

In [53]:
#전처리
ner_df.entity_list=ner_df.entity_list.astype(str)
ner_df.entity_list=ner_df['entity_list'].str.replace('[',"",regex=True)
ner_df.entity_list=ner_df['entity_list'].str.replace(']',"",regex=True)
ner_df.entity_list=ner_df['entity_list'].str.replace('"',"",regex=True)
ner_df.entity_list=ner_df['entity_list'].str.replace('\'',"",regex=True)

In [54]:
#정의한 사전을 활용해 추출된 entity -> ticker 변환
from tqdm import tqdm
last_list=[]
for count in tqdm(range(ner_df.shape[0])):
    change_list=[]
    for i in range(len(ner_df.entity_list[count].split(', '))):
        for key in symbol_dict.keys():
            for j in range(len(symbol_dict[key])):
                if symbol_dict[key][j] == ner_df.entity_list[count].split(', ')[i]:
                    change=ner_df.entity_list[count].split(', ')[i].replace(symbol_dict[key][j],key)
                    change_list.append(change)
    last_list.append(change_list)
                    #ner_df.loc[count,'entity_list']=change_list

        

100%|██████████| 4723/4723 [15:25<00:00,  5.11it/s]  


In [55]:
ner_df['ticker']=last_list

In [56]:
ner_df.ticker=ner_df.ticker.astype(str)
ner_df.ticker=ner_df['ticker'].str.replace('[',"",regex=True)
ner_df.ticker=ner_df['ticker'].str.replace(']',"",regex=True)
ner_df.ticker=ner_df['ticker'].str.replace('"',"",regex=True)
ner_df.ticker=ner_df['ticker'].str.replace('\'',"",regex=True)

In [60]:

#결측치 제거 
ner_df.ticker=ner_df.ticker.replace('',np.nan)

ner_df=ner_df.dropna(subset=['ticker'],how='any').reset_index(drop=True)

In [62]:
ner_df.head(5)

,news_label,contents,entity_list,ticker
0,0,InterContinental Hotels Group PLC (NYSE: IHG) ...,"InterContinental Hotels Group PLC, IHG Hotels ...",IHG
1,1,Citigroup Inc. wants a bigger piece of that de...,Citigroup Inc.,C
2,1,The bank is looking to drum up more business w...,Citigroup,C
3,1,Citigroup even dispatched almost a dozen banke...,Citigroup,C
4,1,"Heading into the financial crisis, Citigroup w...",Citigroup,C


# 랜덤으로 데이터를 섞어서 fine-tuning을 위한 데이터셋 구성을 위해..

In [63]:
ner_df=ner_df.sample(frac=1).reset_index(drop=True)

In [64]:
ner_df.head(5)

,news_label,contents,entity_list,ticker
0,316,"For instance, Generac has worked on finding se...",Generac,GNRC
1,432,Roblox generates revenue from sales of its vir...,Roblox,RBLX
2,46,"After the closure of the deal, Musk is expecte...","Twitter, Reuters",TWTR
3,159,"Tyson earned $2.29 per share, compared to a $1...",Tyson,TSN
4,267,"If you then take into consideration next year,...","Netflix, Aston Martin’",NFLX


In [74]:
ner_df.to_csv('ticker_processing_05_06.csv',encoding='utf-8-sig')

In [83]:
# import pandas as pd

# tt=pd.read_csv('ticker_processing_0623.csv')

# tt=tt.dropna(subset=['ticker'],how='any').iloc[:,2:].reset_index(drop=True)

# papago 번역을 통한 데이터 라벨링 쉽게쉽게

하지만.. 파파고 10000자 제한.. 못쓰겠당

In [65]:
import requests

def get_translate(text):
    client_id = "Client ID" # 개발자센터에서 발급받은 Client ID 값
    client_secret = "Client_secret" # <-- client_secret 기입

    data = {'text' : text,
            'source' : 'en',
            'target': 'ko'}

    url = "https://openapi.naver.com/v1/papago/n2mt"

    header = {"X-Naver-Client-Id":client_id,
              "X-Naver-Client-Secret":client_secret}

    response = requests.post(url, headers=header, data=data)
    rescode = response.status_code

    if(rescode==200):
        send_data = response.json()
        trans_data = (send_data['message']['result']['translatedText'])
        return trans_data
    else:
        print("Error Code:" , rescode)


In [66]:
import pandas as pd
new_ner=pd.read_csv('shuffle_ner_df.csv')

In [67]:
sentiment_dict={'0' : 'neutral','1' : 'positive', '2': 'negative'}
label_list=[]
count=0
for text in new_ner.contents:
    trans_text=get_translate(text)
    print(trans_text)
    print('-------------------------------------------------------------------------------------------------------------------------------------')
    print('텍스트 내 ticker : '+ new_ner.ticker[count])
    print('-------------------------------------------------------------------------------------------------------------------------------------')
    label_num=str(input("라벨을 입력하세요 : "))
    label_text=sentiment_dict[label_num]
    label_list.append(label_text)
    print('-------------------------------------------------------------------------------------------------------------------------------------')
    print(str(count)+'번째 텍스트 라벨은'+label_text+'입니다.')
    count+=1
    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
    

두 회사는 지난해 8월 아마존 문서를 토대로 로이터 조사 결과 미국 회사가 인도 법을 우회하는 데 사용한 클라우드테일 등 소수 판매업체에 수년간 특혜를 줬다는 사실이 드러나면서 2022년 5월 이후 파트너십을 연장하지 않겠다고 발표했다.
-------------------------------------------------------------------------------------------------------------------------------------
텍스트 내 ticker : AMZN
-------------------------------------------------------------------------------------------------------------------------------------
라벨을 입력하세요 : 0
-------------------------------------------------------------------------------------------------------------------------------------
0번째 텍스트 라벨은neutral입니다.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
애플은 또한 내년에 더 큰 버전의 아이패드 프로를 출시할 계획이라고 블룸버그 통신이 보도했다.
-------------------------------------------------------------------------------------------------------------------------------------
텍스트 내 ticker : AAPL
-------------------------------------------------------------------

KeyboardInterrupt: Interrupted by user

In [68]:
label_list

['neutral', 'positive']